<a href="https://colab.research.google.com/github/AllisonOdette0901/Seminario-IdeO/blob/main/RYA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ramificación y acotamiento para el Problema del Agente Viajero

In [ ]:
# paqueterías
import networkx as nx
import math

In [ ]:
g = nx.Graph() # 4 vértices: 0, 1, 2, 3
g.add_edge(0, 1, weight = 1) # arista 1
g.add_edge(0, 2, weight = 1) # arista 2
g.add_edge(0, 3, weight = 10) # arista 3
g.add_edge(1, 2, weight = 5) # arista 4
g.add_edge(1, 3, weight = 2) # arista 5
g.add_edge(2, 3, weight = 3) # arista 6



In [ ]:
# Esta función calcula una cota inferior de la longitud de los ciclos hamiltonianos que comienzan vértices sub_cycle
def lower_bound(g, sub_cycle):
    # calculamos el peso del camino actual
    current_weight = sum([g[sub_cycle[i]][sub_cycle[i + 1]]['weight'] for i in range(len(sub_cycle) - 1)])
    print("sub-ciclo:", sub_cycle, "peso:", current_weight)
    # Creamos como herramienta una nueva gráfica donde guardamos los vértices no utilizados de g
    unused = [v for v in g.nodes() if v not in sub_cycle]
    h = g.subgraph(unused)
    
    # calculamos el peso del árbol de mínima expansión
    t = list(nx.minimum_spanning_edges(h))
    mst_weight = sum([h.get_edge_data(e[0], e[1])['weight'] for e in t])

    # Si la solución actual, sub_cycle es "trivial" (i.e., no contiene vértices o  contiene todos los vértices), entonces
    # proponemos como cota inferior al árbol de expansión mínima y su peso actual
    if len(sub_cycle) == 0 or len(sub_cycle) == g.number_of_nodes():
        return mst_weight + current_weight

    # Si la solución actual sub_cycle no es trivial, entonces podemos sumar el peso de dos aristas que conectan
    # a los vértices de sub_cycle y la parte restante del gráfico
    # Sea "s" el primer vértice de sub_cycle
    s = sub_cycle[0]
    # Sea "t" el último vértice de sub_cycle
    t = sub_cycle[-1]
    # Buscamos una arista de peso mínimo que conecta un vértice distinto de los que estan en sub_cycle a s
    min_to_s_weight = min([g[v][s]['weight'] for v in g.nodes() if v not in sub_cycle])
    # Buscamos una arista de peso mínimo que conecta al vértice t con algún vértice distinto de los que estan en sub_cycle
    min_from_t_weight = min([g[t][v]['weight'] for v in g.nodes() if v not in sub_cycle])

    # Cualquier ciclo que comience con sub_cycle debe tener una longitud:
    # el peso de las aristas de sub_cycle +
    # el mínimo peso de una arista que conecta a sub_cycle y los vertices restantes +
    # el mínimo peso de una arista que conecta a los vertices restantes y sub_cycle 
    return current_weight + min_from_t_weight + mst_weight + min_to_s_weight


# Branch and bound (Ramificación y acotamiento)
# 1. Una gráfica g;
# 2. La solución actual sub_cycle, i.e. several first vertices of cycle under consideration.
# Iniciamos sub_cycle, que está vacio;
# 3. La mejor solución actual es: current_min, no consideramos caminos de mayor peso
# El peso inicial es infinito
def branch_and_bound(g, sub_cycle=None, current_min=float("inf")):
    # si la solución actual es vacía, entonces comenzamos en el vértice 0.
    if sub_cycle is None:
        sub_cycle = [0]

    # Si tenemos todos los vértices de g en el ciclo, entonces calculamos el peso del ciclo y lo retornamos
    if len(sub_cycle) == g.number_of_nodes():
        weight = sum([g[sub_cycle[i]][sub_cycle[i + 1]]['weight'] for i in range(len(sub_cycle) - 1)])
        weight = weight + g[sub_cycle[-1]][sub_cycle[0]]['weight']
        return weight

    # Ahora buscamos todos los nodos los cuales no han sido utilizados en el sub_cyclye
    unused_nodes = list()
    for v in g.nodes():
        if v not in sub_cycle:
          # guardamos (distancia de (v-1,v), vertice)
            unused_nodes.append((g[sub_cycle[-1]][v]['weight'], v))
    
    print("Nodos no utilizados:",unused_nodes)

    # Los ordenamos por la distancia resultante desde el "nodo actual", que es el último nodo de sub_cycle
    unused_nodes = sorted(unused_nodes)

    for (d, v) in unused_nodes:
        assert v not in sub_cycle # Nos aseguramos que efectivamente v no está en sub_cycle
        extended_subcycle = list(sub_cycle)
        extended_subcycle.append(v) # guardamos el nodo v que aun no estaba en la solución
        print('Lista',extended_subcycle)
        # Para cada vértice no utilizado al guardarlo en extended_subcycle, checamos si es un ciclo más corto, si es así lo guardamos
        #print("LWB", lower_bound(g, extended_subcycle))
        if lower_bound(g, extended_subcycle) < current_min:           
            # If there is such a chance, we add the vertex to the current cycle, and proceed recursively.
            new_current_min = branch_and_bound(g, extended_subcycle, current_min)
            if new_current_min < current_min: 
              # actualizamos el ciclo más corto
               current_min = new_current_min
               print("Menor distancia:", current_min)

    # retornamos el ciclo de menor longitud
    return current_min

In [ ]:
branch_and_bound(g)

Nodos no utilizados: [(1, 1), (1, 2), (10, 3)]
Lista [0, 1]
sub-ciclo: [0, 1] peso: 1
Nodos no utilizados: [(5, 2), (2, 3)]
Lista [0, 1, 3]
sub-ciclo: [0, 1, 3] peso: 3
Nodos no utilizados: [(3, 2)]
Lista [0, 1, 3, 2]
sub-ciclo: [0, 1, 3, 2] peso: 6
Menor distancia: 7
Menor distancia: 7
Lista [0, 1, 2]
sub-ciclo: [0, 1, 2] peso: 6
Menor distancia: 7
Lista [0, 2]
sub-ciclo: [0, 2] peso: 1
Lista [0, 3]
sub-ciclo: [0, 3] peso: 10


7



> Ejemplo 2. Con coordenadas



In [ ]:
#  Calculamos la distancia entre dos puntos
def dist(x1, y1, x2, y2):
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
# Función que recibe una lista de 2 tuplas que son las coordenadas
# y retornamos la gráfica correspondiente
def get_graph(coordinates):
    g = nx.Graph()
    n = len(coordinates)
    for i in range(n):
        for j in range(i + 1):
            g.add_edge(i, j, weight=dist(coordinates[i][0], coordinates[i][1], coordinates[j][0], coordinates[j][1]))
    return g

In [ ]:
# red 4
coordenadas = [(174, 25), (129, 99), (268, 212), (211, 209), (156, 82)]
g2 = get_graph(coordenadas)
branch_and_bound(g2)